In [1]:
import fasttext

kbd_model = fasttext.load_model('../data/processed/embeddings/fasttext_skipgram_kbd_100.bin')

In [3]:
import nltk

with open('../data/processed/short_sents.txt', 'r') as f:
    sentences = f.read().split('\n')

sentences

['!..» – мэшхыдэ ар.',
 '!»',
 'Ауэрэ щIымахуэр къэсащ.',
 '!»',
 'КъызжиIэт губгъуэм: «Лажьи шхэж!',
 '!» Си щхьэкIэ мащIэрэ згъэунэхуакъым.',
 '!» Хэт и гугъэнт, Маринэ!',
 '!» жысIэу, сегупсысыжат, къапщтэмэ.',
 '!» жыхуиIэу.',
 '!» –',
 'КъыпфIэщIырт жиIэу Паганини.',
 '!» – Куэдрэ уэ уогубжь.',
 '!» – Къэжэр Пётр и усэ сатырхэм щыщщ.',
 '!» – арат абы и плъэкIэм къикIыр.',
 '!» – гужьеят Къаплъэн-Джэрий.',
 '!» – егупсыст – Далил дэнэ здэкIуар?',
 '!» – егъэщIагъуэ Жыраслъэн.',
 '!» – жаIащ, жылэр къызэщIэвэри.',
 '!» – жари ягъэсыс Алъхъэс цIыкIур.',
 '!» – жиIэри лIыр къыбгъэдыхьащ.',
 '!» – жиIэрт игукIэ Мэлэчыпхъу.',
 '!» – жызоIэ, си жьэр Iурыхуауэ.',
 '!» – жысIащ.',
 '!» – игъэщIэгъуащ абы.',
 '!» – мобыхэм яхуохъущIэ Дин.',
 '!» – уафэм сабэ дрепхъей.',
 '!» – хьэлъэт и гупсысэр.',
 '!»,',
 'Ухэплъэу бжьыхьэ жэщыр зи кууагъ.',
 '( гу лъетэри)',
 'Уоу, Тыргъэтауэ!',
 '( джатэ пцIанэр иIыгъыу къыжьэхохьэ.',
 '( здаунэтIамкIэ маплъэри) Догуэт!',
 '( и псалъэр зэпеудри)',
 'Ущ

In [4]:
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict


def get_clusters(vectors, n_clusters=20):
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agg_clustering.fit_predict(vectors)
    return labels


def get_sents_by_clusters(words, labels):
    sents_by_clusters = defaultdict(list)
    for i, label in enumerate(labels):
        sents_by_clusters[label].append(words[i])

    return sents_by_clusters

In [6]:
import random
from tqdm import tqdm
import os

butch_size = 10000
cluster_num = 1000
cluster_factor = butch_size / cluster_num

for seed in range(111, 120):
    export_path = f'../data/processed/sent_clusters/seed_{seed}/{cluster_factor}_{butch_size}_{cluster_num}'
    os.makedirs(export_path, exist_ok=True)
    
    random.shuffle(sentences)
    
    for offset in tqdm(range(0, len(kbd_model.get_words()), butch_size)):
        butch_sents = sentences[offset:offset + butch_size]
        if len(butch_sents) < cluster_num:
            break
    
        word_vectors = [kbd_model.get_sentence_vector(sent) for sent in tqdm(butch_sents)]
        labels = get_clusters(word_vectors, n_clusters=cluster_num)
        sents_by_clusters = get_sents_by_clusters(butch_sents, labels)
    
        for cluster_label, cluster_sents in sents_by_clusters.items():
            with open(f'{export_path}/cluster_{offset}_{offset + butch_size}_{cluster_label}.txt', 'w') as f:
                f.write('\n'.join(cluster_sents))

 14%|█▍        | 7/50 [00:17<01:45,  2.45s/it]
